# Analyse de Données Nutritionnelles

In [1]:
# coding: utf-8
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
# read the csv file
df = pd.read_csv("foodfacts.csv", sep='\t',low_memory=False)

# Print the shape of df
print(df.shape)

# Print the columns of df
print(df.columns)







(320772, 162)
Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name',
       'generic_name', 'quantity',
       ...
       'ph_100g', 'fruits-vegetables-nuts_100g',
       'collagen-meat-protein-ratio_100g', 'cocoa_100g', 'chlorophyl_100g',
       'carbon-footprint_100g', 'nutrition-score-fr_100g',
       'nutrition-score-uk_100g', 'glycemic-index_100g',
       'water-hardness_100g'],
      dtype='object', length=162)


In [2]:
###Remove Columns with insufficient data (more than 50% of products don't have the value) and columns that won't be retained for the analysis for the analysis
###remove all rows where the nutritional data is insufficient (NaN for all the necessary info) in the curated dataframe

In [3]:
df_edited=df.copy()
for column in df.drop(['categories_fr','categories'], axis=1).columns.values.tolist():
    
    total_rows=df_edited.shape[0]
    nonempty_rows=df_edited[[column]].count().sum()
    if (nonempty_rows/total_rows)<=0.5:
        del df_edited[column]

print(df_edited.columns)

for col in ['url', 'creator', 'created_t', 'created_datetime','last_modified_t', 'last_modified_datetime', 'brands','brands_tags', 'countries', 'countries_tags', 'countries_fr','ingredients_text', 'serving_size', 'additives_n', 'additives','ingredients_from_palm_oil_n','ingredients_that_may_be_from_palm_oil_n','states', 'states_tags', 'states_fr','nutrition-score-uk_100g']:
    del df_edited[col]

df_edited=df_edited.dropna(thresh=3)





    

        




Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name', 'brands',
       'brands_tags', 'categories', 'categories_fr', 'countries',
       'countries_tags', 'countries_fr', 'ingredients_text', 'serving_size',
       'additives_n', 'additives', 'ingredients_from_palm_oil_n',
       'ingredients_that_may_be_from_palm_oil_n', 'nutrition_grade_fr',
       'states', 'states_tags', 'states_fr', 'energy_100g', 'fat_100g',
       'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g',
       'proteins_100g', 'salt_100g', 'sodium_100g', 'nutrition-score-fr_100g',
       'nutrition-score-uk_100g'],
      dtype='object')


In [ ]:
print(df_edited.shape)
print(df_edited.isnull().any())
print(df_edited.columns)
print(df_edited.head())

In [ ]:
plt.figure()
df_edited.plot(kind='bar',x='saturated-fat_100g',y='nutrition-score-fr_100g')
plt.show()